In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import numpy as np

In [ ]:
link = pd.read_csv("links.csv")

### Link Scrapping

In [ ]:
driver = webdriver.Chrome("D:\AIMT course\Term 2\python\chromedriver_win32\chromedriver.exe")

url = "https://www.kijiji.ca/b-real-estate/city-of-toronto/c34l1700273?ll=43.653226%2C-79.383184&address=Toronto%2C+ON&radius=304.0"

def scrape_links():
    link_elements = driver.find_elements('css selector', 'div.info-container div.title a.title')
    links = [link.get_attribute('href') for link in link_elements]
    return links

all_links = []

def click_next():
    try:
        next_button = driver.find_element('xpath', '//div[@class="bottom-bar"]//a[@title="Next"]')
        next_button.click()
    except NoSuchElementException:
        return False

driver.get(url)

for i in range(600):
    current_links = scrape_links()
    all_links.extend(current_links)
    next_page_exists = click_next()


links = pd.DataFrame(all_links)
links.to_csv("links.csv")

driver.quit()

### Details Scrapping

In [ ]:
data_list = []

for url in link["0"]:
    try:
        driver.get(url)
        
        try:
            show_more_button = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'showMoreButton-')]"))
            )

            driver.execute_script("arguments[0].scrollIntoView();", show_more_button)
            driver.execute_script("arguments[0].click();", show_more_button)
            time.sleep(2)

        except TimeoutException:
            continue
        
        page_source = driver.page_source

        soup = BeautifulSoup(page_source, 'html.parser')

        title = soup.find('h1', class_=re.compile(r'title-\d+')).text.strip() if soup.find('h1', class_=re.compile(r'title-\d+')) else np.nan
        address = soup.find('span', {'itemprop': 'address'}).text.strip() if soup.find('span', {'itemprop': 'address'}) else np.nan

        price_value = []
        price_label = []
        price_elements = soup.find_all('div', class_=re.compile(r'priceWrapper-\d+'))
        for price_element in price_elements:
            price_value.append(price_element.find('span').text.strip())
            price_label.append(price_element.find('span', class_=re.compile(r'utilities-\d+')).text.strip())


        date_posted_pattern = re.compile(r'\d+ days ago')
        date_posted_element = soup.find(text=date_posted_pattern)
        date_posted = date_posted_pattern.search(date_posted_element).group(0) if date_posted_element else np.nan

        unit_row = soup.find('div', class_=re.compile(r'unitRow-\d+')).text.strip() if soup.find('div', class_=re.compile(r'unitRow-\d+')) else np.nan

        parking_pattern = re.compile(r'Parking Included:\s*(\d+)')
        parking_element = soup.find(text=parking_pattern)
        parking = parking_pattern.search(parking_element).group(1) if parking_element else np.nan

        agreement_type_pattern = re.compile(r'Agreement Type:\s*(.+)')
        agreement_type_element = soup.find(text=agreement_type_pattern)
        agreement_type = agreement_type_pattern.search(agreement_type_element).group(1) if agreement_type_element else np.nan

        air_conditioning_pattern = re.compile(r'Air Conditioning:\s*(.+)')
        air_conditioning_element = soup.find(text=air_conditioning_pattern)
        air_conditioning = air_conditioning_pattern.search(air_conditioning_element).group(1) if air_conditioning_element else np.nan

        description = soup.find('div', class_=re.compile(r'descriptionContainer-\d+')).text.strip()
    
        visit_element = soup.find('div', class_=re.compile(r'visitCounter-\d+'))
        visit_count = visit_element.find('span').text.strip() if visit_element else np.nan


        data = {
            'Title': title,
            'Address': address,
            'Price Label': price_label,
            'Price Value': price_value,
            'Date Posted': date_posted,
            'Unit Row': unit_row,
            'Parking': parking,
            'Agreement Type': agreement_type,
            'Air Conditioning': air_conditioning,
            'Description': description,
            'Visits': visit_count
        }

        data_list.append(data)
    except TimeoutException:
        print("Timeout while loading the page. Skipping URL:", url)
        continue

driver.quit()

df = pd.DataFrame(data_list)
df.to_csv("final-data-kijiji.csv")